In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from lightgbm import LGBMRegressor
from datetime import datetime, date, timedelta
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [ ]:
train_prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")
train_options = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/options.csv")
train_secondary = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/secondary_stock_prices.csv")
train_financials = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv")
supplemental_prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")
supplemental_options = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/options.csv")
supplemental_financials = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/financials.csv")
stock_list = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv")
train_prices.head()

In [ ]:
stock_list.head()

In [ ]:
feature_cols_x_y = ["Date", "SecuritiesCode", "Open", "High", "Low", "Close", "Volume", "Target"]

feature_cols_x = ["Date", "SecuritiesCode", "Open", "High", "Low", "Close", "Volume"]


feature_cols = ["Date", "SecuritiesCode", "Open", "High", "Low", "Close", "Volume"]

feature_cols_y = ["Date", "Target"]

ndays_1 = 231

def create_data_x_train(df) :    
    #df = df[feature_cols_x].copy()
    df = df[feature_cols_x_y].copy()
    
    df = df.interpolate()
    
    df = df[feature_cols_x].copy()
    
    ndays_2 = 2000
    
    #df = df[df['SecuritiesCode'] == stock_code].reset_index()
    s = df.index[df['Date'] == '2020-12-23'].tolist()
    df = df.loc[s[0] : s[0]+(ndays_1 * ndays_2)]
    df['Date'] = pd.to_datetime(df['Date'])
    #df1 = np.concatenate([df['Open'].values,df['High'],df['Low'],df['Close']])
    df['Date'] = df['Date'].dt.strftime("%Y%m%d").astype(int)
    return df

def create_data_y_train(df) :    
    #df = df[feature_cols_y].copy()
    df = df[feature_cols_x_y].copy()
    
    df = df.interpolate()
    
    df = df[feature_cols_y].copy()
    
    ndays_2 = 2000
    
    #df = df[df['SecuritiesCode'] == stock_code].reset_index()
    s = df.index[df['Date'] == '2020-12-23'].tolist()
    df = df.loc[s[0] : s[0]+(ndays_1 * ndays_2)]
    df['Date'] = pd.to_datetime(df['Date'])
    #df1 = np.concatenate([df['Open'].values,df['High'],df['Low'],df['Close']])
    df['Date'] = df['Date'].dt.strftime("%Y%m%d").astype(int)
    return df


def create_data(df) :    
    df = df[feature_cols].copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.strftime("%Y%m%d").astype(int)
    return df

#a = train_prices['Date'].nunique()
#print(a)


In [ ]:
X_train = create_data_x_train(train_prices)
y_train = create_data_y_train(train_prices)
y_train = y_train['Target'].copy()
#y_train = train_prices["Target"].copy()
X_train.tail()

In [ ]:
lgb_params = {
    "objective": "regression",
    'metric': 'rmse',
    "n_estimators" : 200,
    "num_leaves" : 60,
    "learning_rate" : 0.1,
    "random_seed" : 10}

In [ ]:
model = LGBMRegressor(**lgb_params)
    
model.fit(X_train, y_train)

In [ ]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

In [ ]:
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    
    display(prices)    
    df = create_data(prices)
        
    sample_prediction["Prediction"] = model.predict(df)
    sample_prediction = sample_prediction.sort_values(by = "Prediction", ascending=False)
    sample_prediction['Rank'] = np.arange(0,2000)
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    sample_prediction.drop(["Prediction"],axis=1)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]].copy()
    env.predict(submission)
    display(submission)